In [1]:
# ! pip install pyoso

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [3]:
# Helper function for inserting arrays into queries
def stringify(arr):
    return "'" + "','".join(arr) + "'"

## Identifying relevant projects

In [4]:
df_projects = client.to_pandas(f"""
WITH cte AS (
  SELECT
    p.project_id,
    p.project_source,
    p.display_name,
    SUM(CASE WHEN m.display_name = 'Gas Fees' AND m.metric_name LIKE '%biannually' THEN tm.amount ELSE 0 END) AS gas_fees
  FROM timeseries_metrics_by_project_v0 AS tm
  JOIN metrics_v0 AS m ON m.metric_id = tm.metric_id
  JOIN projects_v1 AS p ON p.project_id = tm.project_id
  JOIN projects_by_collection_v1 AS pbc ON p.project_id = pbc.project_id
  WHERE p.project_source = 'OP_ATLAS' OR pbc.collection_name = 'optimism'
  GROUP BY 1,2,3
)
SELECT * FROM cte
WHERE gas_fees > 0.1
ORDER BY gas_fees
""")
df_projects.tail()

,project_id,project_source,display_name,gas_fees
426,MSfDkUunLKtfuMRdZuSxuemESQuIWKFCh1O6ySIEUs8=,OP_ATLAS,Account Abstraction - ERC-4337,3765.825176
427,1tglFLCPO1i0woe4TW7pBm1iMqrhlWJ1cv2L/5b0VUk=,OSS_DIRECTORY,Infinitism (ERC-4337) - Account Abstraction,3765.825176
428,Jr4krZ4lp1dMiTRDwmF2R9Hw4mF3rsukcBCM4pnF/fc=,OP_ATLAS,Uniswap on Superchain (Oku),4107.200642
429,pWEQHj4IsxU8lx2EDLJpphYaGAvw/yRps3vgTbVCXjQ=,OSS_DIRECTORY,Uniswap,7948.223536
430,NMwxMeqwLAUieJgJHRyjvE6q/xw/KfE7em6oi9U1aRc=,OSS_DIRECTORY,Xen Crypto,72861.923681


In [5]:
PROJECT_IDS = list(df_projects['project_id'].unique())
len(PROJECT_IDS)

431

## Get repos from these projects

In [6]:
df_repos = client.to_pandas(f"""
    SELECT DISTINCT
        artifact_namespace AS github_org,
        artifact_url AS url,
        language,
        is_fork,
        star_count,
        updated_at
    FROM repositories_v0
    WHERE project_id IN ({stringify(PROJECT_IDS)})
    ORDER BY 1 ASC, 6 DESC
""")
df_repos.tail()

,github_org,url,language,is_fork,star_count,updated_at
9574,zkbob,https://github.com/zkbob/load_runner,Rust,False,1,2022-04-15 17:05:08.000 UTC
9575,zkbob,https://github.com/zkbob/trusted-setup-server,,True,0,2022-03-30 16:19:37.000 UTC
9576,zkbob,https://github.com/zkbob/pool-evm-single-l1,Solidity,True,0,2022-01-10 17:17:45.000 UTC
9577,zkcodex,https://github.com/zkcodex/zkcodex,JavaScript,False,0,2024-10-09 19:36:10.000 UTC
9578,zwergdev,https://github.com/zwergdev/whale-bridge,Solidity,False,1,2024-07-01 10:16:30.000 UTC


In [8]:
#df_repos.to_csv("OnchainBuilderRepos.csv")

## Identify relevant packages

In [11]:
df_packages = client.to_pandas(f"""

SELECT DISTINCT
    po.package_owner_artifact_namespace AS repo_owner,
    po.package_owner_artifact_name AS repo_name,
    po.package_artifact_name AS package_name,
    po.package_artifact_source AS package_type
FROM package_owners_v0 AS po
JOIN projects_v1 p ON po.package_owner_project_id = p.project_id
WHERE p.project_source = 'OP_ATLAS'
ORDER BY 1,2

""")

df_packages.tail()

,repo_owner,repo_name,package_name,package_type
724,wighawag,hardhat-preprocessor,hardhat-preprocessor,NPM
725,wighawag,named-logs,named-logs,NPM
726,wighawag,pwag,pwag,NPM
727,wighawag,transfer-gateway,ethereum-transfer-gateway,NPM
728,zemse,hardhat-tracer,hardhat-tracer,NPM


In [12]:
#df_packages.to_csv('DevtoolingPackages.csv')